In [1]:
from bloc import *

In [18]:
key = np.random.randint(0, 2**64, dtype=np.uint64)
print(key)

9586083963649941768


In [26]:
m = np.random.randint(0, 2**64, dtype=np.uint64)
print(f"{m:016x}")
c = block_encrypt(key=key, message=m, rounds=16)
print(f"{c:016x}")
print(f"{block_decrypt(key=key, cipher=c, rounds=16):016x}")

91a21ab21205df3f
3f12c4764f25bbc3
91a21ab21205df3f
